In [1]:
import re
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.font_manager as fm
from matplotlib import rcParams
import seaborn as sns
import matplotlib as mpl
from matplotlib.font_manager import FontProperties


In [2]:
# 新增函数：从文本中提取表格数据
def extract_table_from_text(text):
    """从Markdown文本中提取模型结果表格数据"""
    # 查找Markdown表格部分
    table_pattern = r"\|\s*变量\s*\|\s*变量类型\s*\|\s*系数\s*\|\s*p值\s*\|\s*归一化系数\s*\|\s*归一化百分比\s*\|\s*显著性\s*\|([\s\S]+?)(?:\n\n|\Z)"
    table_match = re.search(table_pattern, text)
    
    if not table_match:
        print("无法识别表格格式")
        return pd.DataFrame()  # 返回空DataFrame
    
    table_content = table_match.group(1)
    
    # 分割每一行，跳过表头分隔行 (|------|------- 这种形式的行)
    rows = []
    for line in table_content.strip().split('\n'):
        line = line.strip()
        if line and not re.match(r"\|\s*-+\s*\|\s*-+", line):
            rows.append(line)
    
    # 检查是否有数据行
    if not rows:
        return pd.DataFrame()  # 返回空DataFrame
    
    # 准备数据列表
    data = []
    columns = ['变量', '变量类型', '系数', 'p值', '归一化系数', '归一化百分比', '显著性']
    
    for row in rows:
        # 分割每个单元格，去除前后的|和空白
        cells = [cell.strip() for cell in row.split('|')]
        # 移除空字符串和首尾空项
        cells = [cell for cell in cells if cell]
        
        if len(cells) >= 7:  # 确保有足够的列
            data.append(cells)
        else:
            print(f"警告: 行格式不正确: {row}")
    
    # 检查是否有有效数据
    if not data:
        return pd.DataFrame()  # 返回空DataFrame
    
    # 创建DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    # 转换数值列
    df['系数'] = df['系数'].astype(float)
    df['p值'] = df['p值'].astype(float)
    df['归一化系数'] = df['归一化系数'].astype(float)
    
    # 处理百分比列
    df['归一化百分比'] = df['归一化百分比'].str.replace('%', '', regex=False).astype(float)
    
    return df

# 改进的中文字体配置
def setup_chinese_font():
    """配置matplotlib支持中文字体的改进版本"""
    # 常见中文字体列表
    chinese_fonts = [
        'SimHei', 'Microsoft YaHei', 'STSong', 'SimSun', 'KaiTi', 'NSimSun',
        'FangSong', 'SimSun-ExtB', 'Microsoft JhengHei',
        'DengXian', 'Source Han Sans CN', 'Source Han Serif CN',
        'WenQuanYi Micro Hei', 'WenQuanYi Zen Hei', 'Noto Sans CJK SC',
        'Noto Sans SC', 'Noto Serif CJK SC', 'Noto Serif SC'
    ]
    
    # 检查系统中可用的字体
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    print("检测到以下可能支持中文的字体:")
    chinese_available = []
    
    for font in chinese_fonts:
        if font in available_fonts:
            chinese_available.append(font)
            print(f" - {font}")
    
    # 如果找到中文字体，配置matplotlib使用它
    if chinese_available:
        font_name = chinese_available[0]  # 使用第一个找到的字体
        plt.rcParams['font.family'] = ['sans-serif']
        plt.rcParams['font.sans-serif'] = [font_name] + plt.rcParams['font.sans-serif']
        plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
        print(f"主要中文字体设置为: {font_name}")
        
        # 创建并返回字体属性对象供稍后使用
        chinese_font_prop = FontProperties(fname=fm.findfont(FontProperties(family=font_name)))
        return chinese_font_prop
    else:
        # 尝试直接找出系统中的中文字体
        system_fonts = []
        for font in available_fonts:
            # 检查字体名称中是否包含可能的中文字体关键词
            if any(keyword in font.lower() for keyword in ['chinese', 'cjk', 'simhei', 'simsun', 'microsoft', 'wqy', 'wenquanyi', 'noto']):
                system_fonts.append(font)
                print(f" - {font} (通过关键词检测)")
        
        if system_fonts:
            font_name = system_fonts[0]
            plt.rcParams['font.family'] = ['sans-serif']
            plt.rcParams['font.sans-serif'] = [font_name] + plt.rcParams['font.sans-serif']
            plt.rcParams['axes.unicode_minus'] = False
            print(f"使用系统检测到的字体: {font_name}")
            
            chinese_font_prop = FontProperties(fname=fm.findfont(FontProperties(family=font_name)))
            return chinese_font_prop
    
    # 如果还是找不到，尝试使用系统默认
    print("警告: 未找到合适的中文字体，尝试使用系统默认")
    plt.rcParams['font.sans-serif'] = ['DejaVu Sans'] + plt.rcParams['font.sans-serif']
    plt.rcParams['axes.unicode_minus'] = False
    
    # 返回None表示没有特定中文字体
    return None

def create_no_significant_vars_chart(title_prefix, output_path, font_prop):
    """为没有显著变量的情况创建说明图表"""
    plt.figure(figsize=(10, 6))
    plt.text(0.5, 0.5, 
             f"模型中没有发现显著变量\n(p<0.05)",
             fontsize=20, ha='center', va='center',
             bbox=dict(boxstyle="round,pad=1", fc="#f0f0f0", ec="gray", alpha=0.8),
             fontproperties=font_prop)
    
    plt.title(f"{title_prefix}分析结果", fontsize=16, pad=20, fontproperties=font_prop)
    plt.axis('off')  # 隐藏坐标轴
    
    # 添加底部说明
    plt.figtext(0.5, 0.1, 
                "注: 模型中的所有变量均未达到统计显著性(p<0.05)",
                ha='center', fontsize=12, 
                bbox=dict(boxstyle='round,pad=0.5', facecolor='#f5f5f5', alpha=0.5),
                fontproperties=font_prop)
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return True

def create_pie_chart(df, title_prefix, output_path, font_prop):
    """创建归一化系数绝对值的饼图"""
    # 筛选显著的变量
    significant_vars = df[df['显著性'] == '是']
    
    # 只选择自变量进行可视化
    iv_vars = significant_vars[significant_vars['变量类型'] == '自变量']
    
    if iv_vars.empty:
        print(f"{title_prefix}: 没有找到显著的自变量")
        return False
    
    # 计算归一化系数的绝对值
    iv_vars['归一化系数绝对值'] = iv_vars['归一化系数'].abs()
    
    # 设置图形大小
    plt.figure(figsize=(12, 9))
    
    # 设置颜色，根据影响方向不同设置不同颜色
    colors = []
    for coef in iv_vars['系数']:
        if coef > 0:
            colors.append('#4CAF50')  # 绿色表示正向影响
        else:
            colors.append('#F44336')  # 红色表示负向影响
    
    # 创建饼图
    wedges, texts, autotexts = plt.pie(
        iv_vars['归一化系数绝对值'],
        labels=iv_vars['变量'],
        autopct='%1.1f%%',
        startangle=90,
        shadow=True,
        explode=[0.05] * len(iv_vars),  # 稍微分离各扇区
        colors=colors,
        textprops={'fontproperties': font_prop, 'fontsize': 12}
    )
    
    # 为标签和百分比设置样式
    plt.setp(autotexts, size=11, weight='bold', color='white')
    
    # 显式设置文本属性，确保使用中文字体
    for text in texts:
        text.set_fontproperties(font_prop)
    
    # 添加标题和注释
    plt.title(f'{title_prefix}分析: 显著自变量的影响力分布 (饼图)', 
              fontsize=16, pad=20, fontweight='bold', fontproperties=font_prop)
    
    # 添加方向指示
    for i, row in iv_vars.reset_index().iterrows():
        direction = "正向" if row['系数'] > 0 else "负向"
        texts[i].set_text(f"{row['变量']}\n({direction})")
        texts[i].set_fontproperties(font_prop)
    
    # 添加图例说明影响方向
    red_patch = plt.Rectangle((0, 0), 1, 1, fc="#F44336", alpha=0.7)
    green_patch = plt.Rectangle((0, 0), 1, 1, fc="#4CAF50", alpha=0.7)
    plt.legend([red_patch, green_patch], ['负向影响', '正向影响'], 
               loc='lower right', fontsize=12, prop=font_prop)
    
    # 添加脚注
    plt.figtext(0.5, 0.01, 
                f"注: 图表基于归一化系数绝对值, 总共{len(iv_vars)}个显著自变量(p<0.05)",
                ha='center', fontsize=10, 
                bbox=dict(boxstyle='round,pad=0.5', facecolor='#f0f0f0', alpha=0.5),
                fontproperties=font_prop)
    
    plt.axis('equal')  # 保持饼图为圆形
    plt.tight_layout()
    
    # 保存图片
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return True

def create_horizontal_bar_chart(df, title_prefix, output_path, font_prop):
    """创建水平条形图，更直观地展示变量影响力"""
    # 筛选显著的变量
    significant_vars = df[df['显著性'] == '是']
    
    # 只选择自变量进行可视化
    iv_vars = significant_vars[significant_vars['变量类型'] == '自变量']
    
    if iv_vars.empty:
        print(f"{title_prefix}: 没有找到显著的自变量")
        return False
    
    # 计算归一化系数的绝对值并排序
    iv_vars['归一化系数绝对值'] = iv_vars['归一化系数'].abs()
    iv_vars = iv_vars.sort_values(by='归一化系数绝对值', ascending=True)  # 升序排列以便在图表中从上到下增大
    
    # 设置图形大小
    plt.figure(figsize=(12, max(6, len(iv_vars) * 0.8)))  # 动态调整高度
    
    # 设置颜色，根据影响方向不同设置不同颜色
    colors = []
    for coef in iv_vars['系数']:
        if coef > 0:
            colors.append('#4CAF50')  # 绿色表示正向影响
        else:
            colors.append('#F44336')  # 红色表示负向影响
    
    # 创建水平条形图
    bars = plt.barh(
        iv_vars['变量'],
        iv_vars['归一化系数绝对值'],
        color=colors,
        height=0.6,
        alpha=0.8
    )
    
    # 添加数值标签
    for i, bar in enumerate(bars):
        row = iv_vars.iloc[i]
        direction = "+" if row['系数'] > 0 else "-"
        plt.text(
            bar.get_width() + 0.01,
            bar.get_y() + bar.get_height()/2,
            f"{direction}{row['归一化系数绝对值']:.2f} ({row['归一化百分比']:.1f}%)",
            va='center',
            fontsize=11,
            fontweight='bold',
            fontproperties=font_prop
        )
    
    # 添加标题和坐标轴标签
    plt.title(f'{title_prefix}分析: 显著自变量的影响力分布', 
             fontsize=16, pad=20, fontweight='bold', fontproperties=font_prop)
    plt.xlabel('归一化系数绝对值', fontsize=12, fontproperties=font_prop)
    
    # 设置Y轴标签的字体 
    plt.yticks(fontproperties=font_prop, fontsize=12)
    
    # 设置网格和背景
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.gca().set_facecolor('#f8f8f8')
    
    # 添加图例说明影响方向
    red_patch = plt.Rectangle((0, 0), 1, 1, fc="#F44336", alpha=0.7)
    green_patch = plt.Rectangle((0, 0), 1, 1, fc="#4CAF50", alpha=0.7)
    plt.legend([green_patch, red_patch], ['正向影响', '负向影响'], 
               loc='lower right', fontsize=12, prop=font_prop)
    
    # 添加脚注
    plt.figtext(0.5, 0.01, 
                f"注: 图表基于归一化系数绝对值, 总共{len(iv_vars)}个显著自变量(p<0.05)",
                ha='center', fontsize=10, 
                bbox=dict(boxstyle='round,pad=0.5', facecolor='#f0f0f0', alpha=0.5),
                fontproperties=font_prop)
    
    plt.tight_layout(pad=3)
    
    # 保存图片
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return True

def create_lollipop_chart(df, title_prefix, output_path, font_prop):
    """创建棒棒糖图，展示变量影响力"""
    # 筛选显著的变量
    significant_vars = df[df['显著性'] == '是']
    
    # 只选择自变量进行可视化
    iv_vars = significant_vars[significant_vars['变量类型'] == '自变量']
    
    if iv_vars.empty:
        print(f"{title_prefix}: 没有找到显著的自变量")
        return False
    
    # 计算归一化系数的绝对值并排序
    iv_vars['归一化系数绝对值'] = iv_vars['归一化系数'].abs()
    iv_vars = iv_vars.sort_values(by='归一化系数绝对值', ascending=True)
    
    # 设置图形大小
    plt.figure(figsize=(12, max(6, len(iv_vars) * 0.8)))
    
    # 创建颜色列表
    positive_color = '#4CAF50'  # 绿色表示正向影响
    negative_color = '#F44336'  # 红色表示负向影响
    
    # 绘制水平线
    for i, row in iv_vars.iterrows():
        y_pos = row['变量']
        x_pos = row['归一化系数绝对值']
        color = positive_color if row['系数'] > 0 else negative_color
        
        # 绘制水平线
        plt.plot([0, x_pos], [y_pos, y_pos], color=color, alpha=0.6, linewidth=2)
        
        # 绘制端点圆圈
        plt.scatter(x_pos, y_pos, s=200, color=color, alpha=0.8, zorder=3)
        
        # 添加数值标签
        direction = "+" if row['系数'] > 0 else "-"
        plt.text(
            x_pos + 0.01,
            y_pos,
            f"{direction}{x_pos:.2f} ({row['归一化百分比']:.1f}%)",
            va='center',
            fontsize=11,
            fontweight='bold',
            fontproperties=font_prop
        )
    
    # 添加标题和坐标轴标签
    plt.title(f'{title_prefix}分析: 显著自变量的影响力分布 (棒棒糖图)', 
             fontsize=16, pad=20, fontweight='bold', fontproperties=font_prop)
    plt.xlabel('归一化系数绝对值', fontsize=12, fontproperties=font_prop)
    
    # 设置Y轴标签的字体大小
    plt.yticks(fontproperties=font_prop, fontsize=12)
    
    # 设置网格和背景
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.gca().set_facecolor('#f8f8f8')
    
    # 设置X轴范围从0开始
    plt.xlim(0, iv_vars['归一化系数绝对值'].max() * 1.15)
    
    # 添加图例说明影响方向
    red_patch = plt.scatter([], [], s=100, color=negative_color)
    green_patch = plt.scatter([], [], s=100, color=positive_color)
    plt.legend([green_patch, red_patch], ['正向影响', '负向影响'], 
               loc='lower right', fontsize=12, prop=font_prop)
    
    # 添加脚注
    plt.figtext(0.5, 0.01, 
                f"注: 图表基于归一化系数绝对值, 总共{len(iv_vars)}个显著自变量(p<0.05)",
                ha='center', fontsize=10, 
                bbox=dict(boxstyle='round,pad=0.5', facecolor='#f0f0f0', alpha=0.5),
                fontproperties=font_prop)
    
    plt.tight_layout(pad=3)
    
    # 保存图片
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return True

def process_all_files():
    """处理所有符合模式的文件并生成图表"""
    # 设置中文字体
    font_prop = setup_chinese_font()
    
    # 设置seaborn样式
    sns.set_style("whitegrid")
    
    # 指定子文件夹路径
    subfolder = "生成结果/social_media/"
    
    # 查找子文件夹中所有符合模式的文件
    file_pattern = os.path.join(subfolder, "*-归一化的模型结果.txt")
    result_files = glob.glob(file_pattern)
    
    if not result_files:
        print(f"没有找到符合模式'{file_pattern}'的文件")
        # 检查子文件夹是否存在
        if not os.path.exists(subfolder):
            print(f"子文件夹 '{subfolder}' 不存在！")
        return
    
    print(f"在'{subfolder}'中找到 {len(result_files)} 个文件待处理")
    
    # 跟踪成功与失败
    success_count = 0
    failure_count = 0
    
    # 处理每个文件
    for file_path in result_files:
        # 提取文件名前缀
        prefix = os.path.basename(file_path).split('-')[0]
        print(f"\n正在处理 {file_path} (前缀: {prefix})...")
        
        try:
            # 读取文件内容
            with open(file_path, 'r', encoding='utf-8') as file:
                result_text = file.read()
            
            # 提取表格数据
            df = extract_table_from_text(result_text)
            
            if df is None:
                # 无法解析文件格式
                print(f"✗ 无法从 {file_path} 提取数据格式")
                failure_count += 1
                continue
                
            # 基本输出路径(不含扩展名)
            output_base = os.path.join(os.path.dirname(file_path), f"{prefix}")
            
            if df.empty:
                # 没有显著变量的情况
                output_path = f"{output_base}-无显著变量.png"
                if create_no_significant_vars_chart(prefix, output_path, font_prop):
                    print(f"✓ 创建了'无显著变量'图表: {output_path}")
                    success_count += 1
                else:
                    failure_count += 1
            else:
                # 有显著变量，创建三种图表
                charts_created = 0
                
                # 1. 创建条形图 (主要图表)
                bar_path = f"{output_base}-条形图.png"
                if create_horizontal_bar_chart(df, prefix, bar_path, font_prop):
                    print(f"✓ 成功创建条形图: {bar_path}")
                    charts_created += 1
                
                # 2. 创建棒棒糖图
                lollipop_path = f"{output_base}-棒棒糖图.png"
                if create_lollipop_chart(df, prefix, lollipop_path, font_prop):
                    print(f"✓ 成功创建棒棒糖图: {lollipop_path}")
                    charts_created += 1
                
                # 3. 创建饼图
                pie_path = f"{output_base}-饼图.png"
                if create_pie_chart(df, prefix, pie_path, font_prop):
                    print(f"✓ 成功创建饼图: {pie_path}")
                    charts_created += 1
                
                if charts_created > 0:
                    success_count += 1
                else:
                    failure_count += 1
                    
        except Exception as e:
            print(f"✗ 处理 {file_path} 时出错: {str(e)}")
            failure_count += 1
    
    # 输出总结
    print(f"\n处理完成: 成功 {success_count} 个, 失败 {failure_count} 个")

# 执行主程序
process_all_files()

检测到以下可能支持中文的字体:
 - SimHei
 - Microsoft YaHei
 - STSong
 - SimSun
 - KaiTi
 - FangSong
 - SimSun-ExtB
 - Microsoft JhengHei
 - DengXian
 - Noto Sans SC
 - Noto Serif SC
主要中文字体设置为: SimHei
在'生成结果/social_media/'中找到 5 个文件待处理

正在处理 生成结果/social_media\Comment-归一化的模型结果.txt (前缀: Comment)...
Comment: 没有找到显著的自变量
Comment: 没有找到显著的自变量
Comment: 没有找到显著的自变量

正在处理 生成结果/social_media\Like-归一化的模型结果.txt (前缀: Like)...
Like: 没有找到显著的自变量
Like: 没有找到显著的自变量
Like: 没有找到显著的自变量

正在处理 生成结果/social_media\Repost-归一化的模型结果.txt (前缀: Repost)...
Repost: 没有找到显著的自变量
Repost: 没有找到显著的自变量
Repost: 没有找到显著的自变量

正在处理 生成结果/social_media\Share-归一化的模型结果.txt (前缀: Share)...
Share: 没有找到显著的自变量
Share: 没有找到显著的自变量
Share: 没有找到显著的自变量

正在处理 生成结果/social_media\View-归一化的模型结果.txt (前缀: View)...
✓ 成功创建条形图: 生成结果/social_media\View-条形图.png


C:\Users\admin\AppData\Local\Temp\ipykernel_24540\3727678518.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iv_vars['归一化系数绝对值'] = iv_vars['归一化系数'].abs()
C:\Users\admin\AppData\Local\Temp\ipykernel_24540\3727678518.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iv_vars['归一化系数绝对值'] = iv_vars['归一化系数'].abs()


✓ 成功创建棒棒糖图: 生成结果/social_media\View-棒棒糖图.png


C:\Users\admin\AppData\Local\Temp\ipykernel_24540\3727678518.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iv_vars['归一化系数绝对值'] = iv_vars['归一化系数'].abs()


✓ 成功创建饼图: 生成结果/social_media\View-饼图.png

处理完成: 成功 1 个, 失败 4 个
